## *Spotify recommender end-to-end*

![diagrama de bloques](./custom_spotify_recommender.png)

### 1. Recolección de datos con la API de Spotify

#### Configuración de la API

In [1]:
# Librerías requeridas
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

Inicialmente se debe configurar una aplicación en el *dashboard* de Spotify, según el [Web API tutorial](https://developer.spotify.com/documentation/web-api/quick-start/):

1. Ir a [developer.spotify.com](https://developer.spotify.com/)
2. Ir a la pestaña *DASHBOARD*
3. Ingresar con el nombre de usuario y contraseña de Spotify
4. Crear una nueva aplicación (*CREATE AN APP*), definiendo un nombre y agregando una descripción
5. *EDIT SETTINGS* -> Redirect URIs -> http://localhost:8080. Requerido para la autenticación desde Python
6. *USERS AND ACCESS* -> *ADD NEW USER* -> Definir nombre y correo electrónico de la cuenta de Spotify. Este será el usuario que podrá acceder a la API desde Python


Con la aplicación creada y configurada, volver al *DASHBOARD* y copiar los campos *Client ID* (el identificador de la aplicación) y *Client Secret* (la clave que permite acceder a la aplicación), además del nombre de usuario con acceso autorizado:

In [2]:
cid = '923e41f1eb3f4f638b102398bb2bee6b'
secret = 'fc0ad2c973de49b99ce16d84e86f0f14'
username = '31prj7pij5zeghgoai4bws4bxshm'
redirect_uri = 'http://localhost:8080'

#### Conexión spotipy-app

In [5]:
# Privilegios: 'user-top-read' para current_user_top_tracks;
scope = 'user-top-read, playlist-modify-private, playlist-modify-public, user-library-modify'
# scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username,scope,cid,secret,redirect_uri)
sp = spotipy.Spotify(auth=token, requests_timeout=120)

#### Listado de top-20 tracks                              

Como punto de partida se usarán los [user's top tracks](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-users-top-artists-and-tracks) que requiere privilegios tipo `user-top-read`.

Otros puntos de partida sugeridos:
- `sp.user_current_followed_artists`
- `sp.current_user_playlists`
- `sp.current_user_recently_played`
- `sp.current_user_top_artists`

entre otros.

In [136]:
top20 = sp.current_user_top_tracks(time_range='short_term', limit=20)

In [137]:
# top20 = sp.current_user_recently_played()

In [138]:
top20

{'items': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2CQHyfluB4mliyv193Qn2L'},
      'href': 'https://api.spotify.com/v1/artists/2CQHyfluB4mliyv193Qn2L',
      'id': '2CQHyfluB4mliyv193Qn2L',
      'name': 'David Rees',
      'type': 'artist',
      'uri': 'spotify:artist:2CQHyfluB4mliyv193Qn2L'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
   

In [139]:
# El resultado está almacenado en un diccionario en donde 'items'
# contiene la información de los tracks

for i, item in enumerate(top20['items']):
    print(i+1, item['name'], '//', item['artists'][0]['name'])

1 De Ellos Aprendí // David Rees
2 Me Voy Enamorando // Chino & Nacho
3 Trascender // Mauricio Alen
4 Hello // Adele
5 Speechless (Full) // Naomi Scott
6 Sonrisas // Pascal
7 Perfect // Ed Sheeran
8 ABRAKADABRA // Ami Rodriguezz
9 Gracias // Los Polinesios
10 Savage Love (Laxed - Siren Beat) // Jawsh 685
11 See You Again (feat. Charlie Puth) // Wiz Khalifa
12 Frío, Frío - En Vivo Estadio Olímpico De República Dominicana/2012 // Juan Luis Guerra 4.40
13 Hasta la Raíz // Natalia Lafourcade
14 La Partida // Antrax
15 Rude // MAGIC!
16 A Thousand Years // Christina Perri
17 My Heart Will Go On - Love Theme from "Titanic" // Céline Dion
18 Contigo Quiero Estar // Skabeche


#### Dataset del preferencias del usuario

Con estos tracks crearemos un dataset (en Pandas) que contendrá, por cada pista, su identificador y sus [características sonoras]():

In [140]:
import pandas as pd

# Extraer ids y nombres de las canciones, y extraer audio_features
tracks = top20['items']
track_ids = []
track_names = []
features = []

for track in tracks:
    track_id = track['id']
    track_name = track['name']
    audio_features = sp.audio_features(track_id)
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])
    
top20_df = pd.DataFrame(features,index = track_names)

In [141]:
top20_df.index

Index(['De Ellos Aprendí', 'Me Voy Enamorando', 'Trascender', 'Hello',
       'Speechless (Full)', 'Sonrisas', 'Perfect', 'ABRAKADABRA', 'Gracias',
       'Savage Love (Laxed - Siren Beat)',
       'See You Again (feat. Charlie Puth)',
       'Frío, Frío - En Vivo Estadio Olímpico De República Dominicana/2012',
       'Hasta la Raíz', 'La Partida', 'Rude', 'A Thousand Years',
       'My Heart Will Go On - Love Theme from "Titanic"',
       'Contigo Quiero Estar'],
      dtype='object')

In [142]:
top20_df.sample()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
"Frío, Frío - En Vivo Estadio Olímpico De República Dominicana/2012",0.574,0.879,11,-2.336,1,0.0387,0.753,0.000004,0.952,0.499,125.284,audio_features,2j5dy9SzXdQ71Y2jgtiFAJ,spotify:track:2j5dy9SzXdQ71Y2jgtiFAJ,https://api.spotify.com/v1/tracks/2j5dy9SzXdQ7...,https://api.spotify.com/v1/audio-analysis/2j5d...,235880,4


In [143]:
top20_df.loc['Frío, Frío - En Vivo Estadio Olímpico De República Dominicana/2012']

danceability                                                    0.574
energy                                                          0.879
key                                                                11
loudness                                                       -2.336
mode                                                                1
speechiness                                                    0.0387
acousticness                                                    0.753
instrumentalness                                             0.000004
liveness                                                        0.952
valence                                                         0.499
tempo                                                         125.284
type                                                   audio_features
id                                             2j5dy9SzXdQ71Y2jgtiFAJ
uri                              spotify:track:2j5dy9SzXdQ71Y2jgtiFAJ
track_href          

In [144]:
# Reorganizar columnas
top20_df = top20_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [145]:
top20_df.shape

(18, 13)

#### Tracks candidatas

Partiendo de los top-20 tracks:
1. Extraeremos los artistas correspondientes (sin repeticiones)
2. Se usará la API para ampliar este listado a:
    - Artistas relacionados (`sp.artist_related_artists`)
    - Artistas con nuevos lanzamientos (`sp.new_releases`) para agregar "novedad" al playlist generado

Con este listado más amplio de artistas, obtenido en (1) y (2) se obtendrá:

3. El listado de albums
4. Y por cada album el listado de tracks

Y finalmente, para el listado resultante se obtendrán las *audio features*. El dataset resultante serán las pistas candidatas, que luego se llevarán al sistema de recomendación

In [146]:
# 1. Extraer los artistas correspondientes a las top-20 tracks (sin repeticiones)
ids_artists = []
print('Artistas en mi top20:')
print('=====================')
for item in top20['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas en mi top20:
2CQHyfluB4mliyv193Qn2L: David Rees
5NS0854TqZQVoRmJKSWtFZ: Chino & Nacho
2xobqRIT4uGkrZVPJjRQeY: Mauricio Alen
4dpARuHxo51G3z768sgnrY: Adele
2Zi3RrdQqk63Xj0914STkS: Naomi Scott
2fuu7gBnfDhgJNt1Yr4ERu: Pascal
6eUKZXaKkcviH0Ku9w2n3V: Ed Sheeran
4P5rq4W7tpIy0Y8THbuUjb: Ami Rodriguezz
089IZ7FwRjpOxPypnAG7kW: Los Polinesios
56mfhUDKa1vec6rSLZV5Eg: Jawsh 685
137W8MRPWKqSmrBGDBFSop: Wiz Khalifa
3nlpTZci9O5W8RsNoNH559: Juan Luis Guerra 4.40
1hcdI2N1023RvSwLzTtdsp: Natalia Lafourcade
1yY5CdhCuocj4GvjOEMFJF: Antrax
0DxeaLnv6SyYk2DOqkLO8c: MAGIC!
7H55rcKCfwqkyDFH9wpKM6: Christina Perri
4S9EykWXhStSc15wEx8QFK: Céline Dion
7cGjXTlsW4AZZEf4HLmYya: Skabeche
Número de artistas (sin repeticiones): 18


In [147]:
# 2.1 Ampliar al listado anterior a artistas relacionados
# Por cada artista que sigo y que está en mi top20 buscar artistas similares y añadirlos al listado
print('')
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id)

ids_artists.extend(ids_similar_artists)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas similares:
6aZyMrc4doVtZyKNilOmwu: Colbie Caillat
2Sqr0DXoaYABbjBo9HaMkM: Sara Bareilles
4utLUGcTvOJFr6aqIJtYWV: Skylar Grey
7h4j9YTJJuAHzLCc3KCvYu: Kina Grannis
5xKp3UyavIBUsGy3DQdXeF: A Great Big World
2TL8gYTNgD6nXkyuUdDrMg: Jasmine Thompson
58MLl9nC29IXbE4nEtuoP2: Alex & Sierra
5lKZWd6HiSCLfnDGrq9RAm: Leona Lewis
2PCUhxD40qlMqsKHjTZD2e: Parachute
3w6zswp5THsSKYLICUbDTZ: Gabrielle Aplin
6p5JxpTc7USNnBnLzctyd4: Phillip Phillips
2vm8GdHyrJh2O2MfbQFYG0: Ingrid Michaelson
4phGZZrJZRo4ElhRtViYdl: Jason Mraz
3BmGtnKgCSGYIUhmivXKWX: Kelly Clarkson
3QLIkT4rD2FMusaqmkepbq: Rachel Platten
2gsggkzM5R49q6jpPvazou: Jessie J
2AQjGvtT0pFYfxR3neFcvz: Jordin Sparks
2WX2uTcsvV5OnS0inACecP: Birdy
0Cav8jyZKAHMFbAusOmjku: Christina Grimmie
7FgMLbnZVrEnir95O0YujA: Five For Fighting
2C1zgkYFPzuU7GBM66c1S9: Victor J Sefo
0ZVAv3drBuIRSc88ATH6UK: DJ Noiz
6UVVGhDsgd1Nh6vQcIERIb: Jaro Local
1dXyyDButMVteLKOOVqz6Z: Eduardo Luzquiños
0q4NrXqJnc367PieejuROJ: Vice
1OkPoEn52lUQfSEmPw45rT: Jason Demarco
3X

In [148]:
# 2.2 Ampliar el listado anterior con artistas con nuevos lanzamientos

print('')
print('Artistas con nuevos lanzamientos:')
print('=====================')
new_releases = sp.new_releases(limit=20)['albums']
for item in new_releases['items']:
    artist_id = item['artists'][0]['id']   #[0] porque puede haber varios artistas, se tomará el primero
    artist_name = item['artists'][0]['name']
    album_name = item['name']   # Nombre del album, puramente informativo
    release_date = item['release_date'] # Fecha de lanzamiento, puramente informativo
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas con nuevos lanzamientos:
790FomKkXshlbRYZFtlgla: KAROL G - // MAÑANA SERÁ BONITO, 2023-02-24
4vhNDa5ycK0ST968ek7kRr: Carlos Vives - // El club de los Graves (Banda Sonora Original), 2023-02-22
3AA28KZvwAUcZuOKwyblJQ: Gorillaz - // Cracker Island, 2023-02-24
7bWN0FHvLppK8ozEH6exdi: M2H - // Homies & Beaches, 2023-02-22
7vXDAI8JwjW531ouMGbfcp: TINI - // Cupido, 2023-02-16
1G89WXRVVAEjU4VIwgg6XD: Ventino - // Ventino: El Precio De La Gloria (Banda Sonora Original De La Serie De Televisión), 2023-02-13
39yVoqm6sYFvvqF1RciUVf: Carlos Rivera - // Sincerándome, 2023-02-17
2wY79sveU1sp5g7SokKOiI: Sam Smith - // Gloria, 2023-01-27
0eHQ9o50hj6ZDNBt6Ys1sD: Yandel - // Resistencia, 2023-01-13
2R21vXR83lH98kGeO99Y66: Anuel AA - // LLNM2, 2022-12-09
1DxLCyH42yaHKGK3cl5bvG: Maria Becerra - // LA NENA DE ARGENTINA, 2022-12-08
2oXKVuZqDv85M1ynjVMp3J: Mabiland - // TORQUE: Vol. 1, 2022-12-06
7tYKF4w9nC0nq9CsPZTHyP: SZA - // SOS, 2022-12-09
2LRoIwlKmHjgvigdNGBHNo: Feid - // SIXDO, 2022-12-02
4S

In [149]:
# Obtener el listado de albums de cada uno de los anteriores artistas.
# Se limitará a únicamente 1 album (limit=1), para evitar tener un listado gigantesco

id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artista {i+1} de {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1) # para evitar tener una lista gigantesca
    for album in albums['items']:
        id_albums.append(album['id'])
print('¡Listo!')

Procesando artista 1 de 348...
Procesando artista 2 de 348...
Procesando artista 3 de 348...
Procesando artista 4 de 348...
Procesando artista 5 de 348...
Procesando artista 6 de 348...
Procesando artista 7 de 348...
Procesando artista 8 de 348...
Procesando artista 9 de 348...
Procesando artista 10 de 348...
Procesando artista 11 de 348...
Procesando artista 12 de 348...
Procesando artista 13 de 348...
Procesando artista 14 de 348...
Procesando artista 15 de 348...
Procesando artista 16 de 348...
Procesando artista 17 de 348...
Procesando artista 18 de 348...
Procesando artista 19 de 348...
Procesando artista 20 de 348...
Procesando artista 21 de 348...
Procesando artista 22 de 348...
Procesando artista 23 de 348...
Procesando artista 24 de 348...
Procesando artista 25 de 348...
Procesando artista 26 de 348...
Procesando artista 27 de 348...
Procesando artista 28 de 348...
Procesando artista 29 de 348...
Procesando artista 30 de 348...
Procesando artista 31 de 348...
Procesando artist

In [150]:
# Por cada album extraer 3 tracks

id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    print(f'Procesando album {i+1} de {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=3)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
print(f'¡Listo! Número total de tracks pre-candidatos: {len(id_tracks)}')

Procesando album 1 de 348...
Procesando album 2 de 348...
Procesando album 3 de 348...
Procesando album 4 de 348...
Procesando album 5 de 348...
Procesando album 6 de 348...
Procesando album 7 de 348...
Procesando album 8 de 348...
Procesando album 9 de 348...
Procesando album 10 de 348...
Procesando album 11 de 348...
Procesando album 12 de 348...
Procesando album 13 de 348...
Procesando album 14 de 348...
Procesando album 15 de 348...
Procesando album 16 de 348...
Procesando album 17 de 348...
Procesando album 18 de 348...
Procesando album 19 de 348...
Procesando album 20 de 348...
Procesando album 21 de 348...
Procesando album 22 de 348...
Procesando album 23 de 348...
Procesando album 24 de 348...
Procesando album 25 de 348...
Procesando album 26 de 348...
Procesando album 27 de 348...
Procesando album 28 de 348...
Procesando album 29 de 348...
Procesando album 30 de 348...
Procesando album 31 de 348...
Procesando album 32 de 348...
Procesando album 33 de 348...
Procesando album 34

In [151]:
# En total se tendrán 543 (890) pistas candidatas. Para cada una se extraerán los
# audio_features y todo se almacenará en un dataframe

track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Procesando track {i+1} de {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    #No incluir pistas sin "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])
print('¡Listo!')

candidatos_df = pd.DataFrame(features,index = track_names)

Procesando track 1 de 890...
Procesando track 2 de 890...
Procesando track 3 de 890...
Procesando track 4 de 890...
Procesando track 5 de 890...
Procesando track 6 de 890...
Procesando track 7 de 890...
Procesando track 8 de 890...
Procesando track 9 de 890...
Procesando track 10 de 890...
Procesando track 11 de 890...
Procesando track 12 de 890...
Procesando track 13 de 890...
Procesando track 14 de 890...
Procesando track 15 de 890...
Procesando track 16 de 890...
Procesando track 17 de 890...
Procesando track 18 de 890...
Procesando track 19 de 890...
Procesando track 20 de 890...
Procesando track 21 de 890...
Procesando track 22 de 890...
Procesando track 23 de 890...
Procesando track 24 de 890...
Procesando track 25 de 890...
Procesando track 26 de 890...
Procesando track 27 de 890...
Procesando track 28 de 890...
Procesando track 29 de 890...
Procesando track 30 de 890...
Procesando track 31 de 890...
Procesando track 32 de 890...
Procesando track 33 de 890...
Procesando track 34

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [152]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Intro,3YJEyVQofYTJDOEbfUxhHO,0.1160,0.677,98413,0.611,0.000115,5,0.2480,-6.987,1,0.0601,96.996,0.586
No One,3kIb1b4YbysgOiIUxd93RL,0.0824,0.581,219867,0.779,0.000000,11,0.1590,-3.657,0,0.0435,119.954,0.308
I'll Be There,4r1CoAATVuxag55Ct3Y1aX,0.0288,0.627,194387,0.858,0.000000,7,0.1990,-3.116,1,0.0394,100.074,0.399
Trap de Free Fire,6hhI7D9oJSpWTIOcHrKSI5,0.0177,0.884,372295,0.878,0.000118,1,0.0909,-6.126,1,0.0814,122.005,0.802
El Mejor Día Del Año,1O4oJxeeu5alzp2MLBDn65,0.1900,0.746,224827,0.885,0.000003,2,0.0380,-5.815,1,0.1100,110.012,0.662


In [153]:
candidatos_df = candidatos_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [154]:
candidatos_df.shape

(888, 13)

In [155]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Intro,3YJEyVQofYTJDOEbfUxhHO,0.1160,0.677,98413,0.611,0.000115,5,0.2480,-6.987,1,0.0601,96.996,0.586
No One,3kIb1b4YbysgOiIUxd93RL,0.0824,0.581,219867,0.779,0.000000,11,0.1590,-3.657,0,0.0435,119.954,0.308
I'll Be There,4r1CoAATVuxag55Ct3Y1aX,0.0288,0.627,194387,0.858,0.000000,7,0.1990,-3.116,1,0.0394,100.074,0.399
Trap de Free Fire,6hhI7D9oJSpWTIOcHrKSI5,0.0177,0.884,372295,0.878,0.000118,1,0.0909,-6.126,1,0.0814,122.005,0.802
El Mejor Día Del Año,1O4oJxeeu5alzp2MLBDn65,0.1900,0.746,224827,0.885,0.000003,2,0.0380,-5.815,1,0.1100,110.012,0.662


In [156]:
top20_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.0,7,0.4390,-6.479,1,0.1160,166.843,0.675
Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.0,10,0.3920,-3.411,1,0.0687,99.957,0.511
Trascender,0Rx0gzyQyKT75fPfpbeEmO,0.5250,0.678,219939,0.877,0.0,11,0.2700,-5.471,1,0.3010,159.983,0.642
Hello,62PaSfnXSMyLshYJrlTuL3,0.3300,0.578,295502,0.430,0.0,5,0.0854,-6.134,0,0.0305,78.991,0.288
Speechless (Full),0XPsOSYzDJZJArevQNm2AR,0.4670,0.474,208801,0.489,0.0,6,0.1190,-7.625,0,0.0342,124.844,0.230


### 2. Sistema de recomendación

![el sistema de recomendación](./sistema_recomendacion.png)

Veamos en detalle el sistema de recomendación:

![el sistema de recomendación en detalle](./sistema_recomendacion_detalle.png)


#### El filtrado basado en contenido

Permite cuantificar qué tan similar es un ítem de `candidatos_df` a un ítem de `top20_df`.

Una forma de hacer esta comparación es usando la similitud del coseno:

![vectores de características](./vectores_caracteristicas.png)

![la similitud del coseno](./similitud_coseno.png)

Calcularemos este similitud entre cada top-20 y cada una de las pistas candidatas (matriz de 20 x n_pistas_candidatas)

In [157]:
# Extraer sólo los features en formato numpy array
top20_mtx = top20_df.iloc[:,1:].values  # "1:" solo toma variables numéricas
candidatos_mtx = candidatos_df.iloc[:,1:].values

In [158]:
from sklearn.preprocessing import StandardScaler

# Estandarizar cada columna de features: mu = 0, sigma = 1
# pues cada característica tiene una escala diferente
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

In [159]:
print(t20_scaled.mean(axis=0))
print(t20_scaled.std(axis=0))

[ 0.00000000e+00 -2.46716228e-17 -2.71387850e-16  2.83723662e-16
  5.85951041e-17 -2.46716228e-17  1.85037171e-17  1.60365548e-16
 -4.31753398e-17 -9.71445147e-17 -3.20731096e-16  1.20274161e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [160]:
import numpy as np

# Normalizar cada vector de características (es decir por filas)

# Magnitudes de cada vector (o pista)
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))

# Normalización
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

print(np.sqrt((t20*t20).sum(axis=1)))
print(np.sqrt((can*can).sum(axis=1)))


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [161]:
from sklearn.metrics.pairwise import linear_kernel

# Calcular similitudes del coseno entre cada top-20 y cada
# una de las pistas candidatas
cos_sim = linear_kernel(t20,can)
cos_sim.shape

(18, 888)

In [162]:
# Ejemplo: ¿qué tanto se parece una pista candidata a una del top-20?
print(cos_sim[6,270])
print(cos_sim[3,24])


-0.034762948914002965
-0.39548052549152724


In [163]:
# Obtener candidatos para una pista dada

# Dada una pista del top-20 (pos = 0, 1, ..., 19) extraer "ncands" candidatos, usando
# "cos_sim" y siempre y cuando superen un umbral de similitud

def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.8):
    # Obtener todas las pistas candidatas por encima de umbral
    idx = np.where(cos_sim[pos,:]>=umbral)[0] # ejm. idx: [27, 82, 135]
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncands", retornar únicamente un total de "ncands"
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [164]:
# Ejemplo de uso
for i in range(18):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i} ==> pistas candidatas: {cands}, similitudes: {cos_sim[i,cands]}')

0 ==> pistas candidatas: [622  58], similitudes: [0.84688358 0.82746389]
1 ==> pistas candidatas: [163 243   2 811 459], similitudes: [0.88113646 0.83597014 0.8344744  0.829697   0.81795397]
2 ==> pistas candidatas: [285], similitudes: [0.82806505]
3 ==> pistas candidatas: [859 747], similitudes: [0.86833058 0.85915595]
4 ==> pistas candidatas: [ 86 540 295], similitudes: [0.84418438 0.82441414 0.8005996 ]
5 ==> pistas candidatas: [433 532], similitudes: [0.85357857 0.81853371]
6 ==> pistas candidatas: [ 48 535 768], similitudes: [0.86938713 0.86237447 0.82313665]
7 ==> pistas candidatas: [525 519  81], similitudes: [0.90002002 0.81103662 0.80920735]
8 ==> pistas candidatas: [230 524 820  70 204], similitudes: [0.92655961 0.88232285 0.88117716 0.84282154 0.84144819]
9 ==> pistas candidatas: [301 715 216], similitudes: [0.84902604 0.82572394 0.80773707]
10 ==> pistas candidatas: [693 790 545 142], similitudes: [0.83836099 0.83455312 0.83284541 0.80505769]
11 ==> pistas candidatas: [734 

#### Creación de playlist con listado de pistas sugeridas

In [165]:
# Para crear la playlist se requieren únicamente los ids
ids_t20 = []
ids_playlist = []

for i in range(top20_df.shape[0]):
    print(top20_df.index[i])   # Nombre de la pista en el top-20
    ids_t20.append(top20_df['id'][i])
    
    # Obtener listado de candidatos para esta pista
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    # Si hay pistas relacionadas obtener los ids correspondientes
    # e imprimir en pantalla
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Obtener los ids correspondientes e imprimir en pantalla
        for j in cands:
            id_cand = candidatos_df['id'][j]
            ids_playlist.append(id_cand)
            
            # E imprimir en pantalla el candidato
            print(f'   {candidatos_df.index[j]}')

De Ellos Aprendí
   The Breeze (Cool) (feat. TRÉ & Wiz Khalifa)
   Homey Girl - 2022 Version
Me Voy Enamorando
   Dancing Queen
   The Remedy (I Won't Worry)
   I'll Be There
   Never Really Over
   All Day (with Tablo)
Trascender
   PERO TÚ
Hello
   Had It All
   Looking For Sade
Speechless (Full)
   We Go Down Together (with Khalid)
   You're The One That I Want
   Sexy Red Dress
Sonrisas
   Tales
   Quieres Munchies?
Perfect
   Caminaré
   Higher Love
   Said I Loved You…But I Lied
ABRAKADABRA
   Not A Love Song
   Diva Yourself
   Plan A
Gracias
   Like The Way It Feels
   A Billion Hits
   Amándote (feat. Jandy Feliz)
   Entendemos
   Shivers
Savage Love (Laxed - Siren Beat)
   Quiero Amborgesa
   Night In The Club
   Savage Love (Laxed – Siren Beat) [BTS Remix]
See You Again (feat. Charlie Puth)
   Love Me More
   Anything
   Perfect For You
   Hold Me While You Wait
Frío, Frío - En Vivo Estadio Olímpico De República Dominicana/2012
   Man With The Bag
   The Riddle (Live)
   Je 

In [166]:
# Eliminar candidatos que ya están en el top-20
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]

# Y eliminar posibles repeticiones
ids_playlist_dep = list(set(ids_playlist_dep))

In [167]:
print(ids_playlist_dep)

['4tYW2956Q93JTDqnLx2oDT', '1xnaDZi0ZSOSDozKFEiobD', '5eqeMVPaAdTguY3X80o7Dn', '3DmQvbSuwGoDJkDm369GEz', '4dpRlEWhhmDnUEXuMAxC4k', '18QfMXWWtbAF03nEPnvu8c', '2Y67qsABsPKMrvCxPCzL6r', '5qOwKwblw4qR3kfKMWSlnn', '6fIEBZB2wtqiyk6Se7qPsK', '48IDve2DfIE5oswLgxWfnz', '31bT87g6Ntoyn6u0SCrIs4', '2Ysi49tRdVH74d1Cnc4o94', '2ETyUK0F39pIYkkoQu6lRu', '7fGqEv1BZ0dxuQqWlYOjli', '73CbJykoV6WapWGfTeRzTl', '3PgnXyzWXrEoE0oQdLMF3M', '5eZcNtcQYhCk1drbSvzkrK', '7bKzJpTnMkRxGyMFNtNEpu', '70vr24gR1A5J2aE3rhV8Lo', '4r1CoAATVuxag55Ct3Y1aX', '2Pz9DnS04JtX70gERPeorz', '6JnMhYegrRKKSp82BqkJGD', '3y5bErXoyFjMNBCkFq6YZS', '1UtGcsbiqHtzxaTe3OgvlG', '1i6JkxHU9mPwjKDCMLvVie', '28Ra14Zs2AiIBGyG7skWWK', '6yTPAzXmuvE5E4J7zfUKIu', '5vQabTPJCm71G9LoVpVIVY', '758vpzWs4cEoia1WQHiSwD', '0CUbGs61barxaLWZkSm8EO', '2qcgamAgC9nYoPqbQ9xXFi', '4TgxFMOn5yoESW6zCidCXL', '6q3186s92a91zx3IG16hBE', '6maG9OGek8E3dEwp0BwFjT', '1n5YR03VSAfuRj9quTGydT', '3U0FNELhr5TV1aSfZW18Py', '0vv1U2vCd7HEFrY4cNSOEj', '0atYGVGJ39BNCh3RzAUtDN', '1IIV8jglA0

In [168]:
len(ids_playlist_dep) # Hemos pasado de ~500 pre-candidatos a muchas menos pistas

56

In [172]:
# Crear el playlist en spotify
pl = sp.user_playlist_create(user = username, 
                            name = 'Spotipy Recommender Playlist',
                            description = 'Playlist creada con el sistema de recomendación (Codificando Bits)')

sp.playlist_add_items(pl['id'],ids_playlist_dep)

{'snapshot_id': 'Myw0OWM4MDJjNTA1MzgyYTdmZmQ5YjQ2OWZkMzgzNTZjYjI5ZjliY2Ix'}